<a href="https://colab.research.google.com/github/ngmars/Burger-Shop-ReactJs/blob/master/FER_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dataset

In [ ]:
!pip install livelossplot==0.1.2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import utils
import os
%matplotlib inline
import livelossplot

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau 
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
#from livelossplot import PlotLossesCallback
import tensorflow as tf
print("Tensorflow verision: ", tf.__version__)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


Tensorflow verision:  2.2.0


Plot Sample Image


In [ ]:
for expression in os.listdir("/content/drive/My Drive/Colab Notebooks/Project/train/"):
  print(str(len(os.listdir("/content/drive/My Drive/Colab Notebooks/Project/train/"+ expression)))+ " " + expression + " Images")

3995 angry Images
4965 neutral Images
3171 surprise Images
436 disgust Images
7234 happy Images
4097 fear Images
4835 sad Images


Generating training and test data


In [ ]:
img_size= 48
batch_size =64


datagen_train= ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("/content/drive/My Drive/Colab Notebooks/Project/train/", 
                                                    target_size=(img_size,img_size),
                                                    color_mode='grayscale',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
datagen_test= ImageDataGenerator(horizontal_flip=True)
test_generator = datagen_test.flow_from_directory("/content/drive/My Drive/Colab Notebooks/Project/test/", 
                                                    target_size=(img_size,img_size),
                                                    color_mode='grayscale',
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

Found 28733 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


Adding CNN layers

In [ ]:
#addition of 4 CNN layers 
model = Sequential()

#1-conv 
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#2-conv 
model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#3-conv 
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#4-conv 
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

Training and evaluating model

In [ ]:
epochs =10
steps_per_epoch= train_generator.n//train_generator.batch_size
test_steps=test_generator.n//test_generator.batch_size
checkpoint= ModelCheckpoint("model_weights.h5", monitor='val_accuracy', save_weights_only=True,
                            mode='max', verbose=1)
reduce_lr= ReduceLROnPlateau(monitor='val_loss', factor= 0.1, patience=2, model='auto',
                              min_lr=0.00001)
callbacks= [ checkpoint, reduce_lr]
history = model.fit(x= train_generator, steps_per_epoch= steps_per_epoch, epochs= epochs, validation_data= test_generator, validation_steps=test_steps, callbacks= callbacks)


Epoch 1/10
448/448 [==============================] - ETA: 0s - loss: 1.7043 - accuracy: 0.3456 
Epoch 00001: saving model to model_weights.h5
448/448 [==============================] - 12110s 27s/step - loss: 1.7043 - accuracy: 0.3456 - val_loss: 1.6615 - val_accuracy: 0.4148 - lr: 5.0000e-04
Epoch 2/10
448/448 [==============================] - ETA: 0s - loss: 1.3978 - accuracy: 0.4689
Epoch 00002: saving model to model_weights.h5
448/448 [==============================] - 1375s 3s/step - loss: 1.3978 - accuracy: 0.4689 - val_loss: 1.3561 - val_accuracy: 0.4893 - lr: 5.0000e-04
Epoch 3/10
448/448 [==============================] - ETA: 0s - loss: 1.2625 - accuracy: 0.5216
Epoch 00003: saving model to model_weights.h5
448/448 [==============================] - 1361s 3s/step - loss: 1.2625 - accuracy: 0.5216 - val_loss: 1.2298 - val_accuracy: 0.5269 - lr: 5.0000e-04
Epoch 4/10
448/448 [==============================] - ETA: 0s - loss: 1.1894 - accuracy: 0.5475
Epoch 00004: saving model

Saving it as Json File

In [ ]:
model_json= model.to_json()
with open("model.json","w") as json_file:
  json_file.write(model_json)
